In [ ]:
import os
import re
import requests
import pandas as pd
import numpy as np
import geopandas as gpd

# ---- NYC Open Data dataset IDs ----
DS_TREE_POINTS = "hn5i-inap"  # Forestry Tree Points (street-tree planting spaces / trees)
DS_WORK_ORDERS = "bdjm-n7q4"  # Forestry Work Orders (events: removals, plantings, etc.)
DS_CENSUS_2015 = "uvpi-gqnh"  #  2015 Street Tree Census - dont knwo if i ineed

# ---- Analysis window ----
YEARS = list(range(2010, 2018))  # 2010..2017 inclusive
AS_OF = {y: pd.Timestamp(f"{y}-12-31") for y in YEARS}

# Spatial CRS for NYC (planar feet) if you do spatial joins later
TARGET_CRS = "EPSG:2263"
BASE = "https://data.cityofnewyork.us/resource/{id}"

### Setting Pandas Options 
pd.options.display.max_columns = 500

In [ ]:
### Defining functions in roder to pull the data as needed. 
def _headers():
    h = {}
    tok = os.getenv("SOCRATA_APP_TOKEN")
    if tok:
        h["X-App-Token"] = tok
    return h

def socrata_fetch_tabular(dataset_id, where=None, select=None, limit=25000, max_rows=None, verbose=True):
    """
    Pull ALL rows (paged) from a Socrata tabular dataset (.json). Returns a DataFrame.
    Set max_rows=None to attempt full download.
    """
    url = f"{BASE.format(id=dataset_id)}.json"
    params_base = {"$limit": limit}
    if where:  params_base["$where"]  = where
    if select: params_base["$select"] = select

    frames, offset, got = [], 0, 0
    while True:
        if max_rows is not None and got >= max_rows:
            break
        page = min(limit, max_rows - got) if max_rows else limit
        p = dict(params_base); p["$offset"] = offset; p["$limit"] = page
        r = requests.get(url, params=p, headers=_headers(), timeout=180); r.raise_for_status()
        rows = r.json()
        if not rows: break
        frames.append(pd.DataFrame(rows))
        n = len(rows); got += n; offset += n
        if verbose: print(f"{dataset_id}: fetched {got:,} rows…")
        if n < page: break
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def socrata_fetch_geojson(dataset_id, limit=50000, verbose=True):
    """
    Pull ALL features from a Socrata GeoJSON endpoint (.geojson). Returns a GeoDataFrame (EPSG:4326).
    Useful for polygon layers like census tracts.
    """
    feats, offset = [], 0
    while True:
        url = f"{BASE.format(id=dataset_id)}.geojson?$limit={limit}&$offset={offset}"
        r = requests.get(url, headers=_headers(), timeout=180); r.raise_for_status()
        gj = r.json(); chunk = gj.get("features", [])
        if not chunk: break
        feats.extend(chunk); offset += len(chunk)
        if verbose: print(f"{dataset_id} (geojson): fetched {offset:,} features…")
        if len(chunk) < limit: break
    return gpd.GeoDataFrame.from_features(feats, crs="EPSG:4326") if feats else gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

def add_lat_lon_from_location(df, location_col="location"):
    """
    Unpack Socrata 'location' Point dict into numeric lat/lon, preserving all other columns.
    """
    if location_col not in df.columns: return df
    def _lat(v):
        if isinstance(v, dict) and "latitude" in v: return v["latitude"]
        if isinstance(v, dict) and "coordinates" in v: return v["coordinates"][1]
    def _lon(v):
        if isinstance(v, dict) and "longitude" in v: return v["longitude"]
        if isinstance(v, dict) and "coordinates" in v: return v["coordinates"][0]
    out = df.copy()
    out["lat"] = pd.to_numeric(out[location_col].map(_lat), errors="coerce")
    out["lon"] = pd.to_numeric(out[location_col].map(_lon), errors="coerce")
    return out

def convert_datetime_columns(df, extra_force=()):
    """
    Convert columns that look like dates/times (by name) to pandas datetime.
    extra_force: iterable of column names to always attempt.
    """
    df = df.copy()
    name_pat = re.compile(r"(date|time|_at|_on)", re.I)
    candidates = [c for c in df.columns if name_pat.search(c)]
    candidates = sorted(set(candidates).union(extra_force))
    for c in candidates:
        df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

def classify_work_orders(works):
    """
    Add boolean flags for 'is_tree_removal' and 'is_tree_planting' + pick a best 'event_date'.
    Uses wotype + wstatus. Tighten patterns later if you discover coded fields.
    """
    w = works.copy()
    def s(x): return x.astype(str).str.lower() if isinstance(x, pd.Series) else pd.Series([], dtype=str)
    wotype, wstatus = s(w.get("wotype")), s(w.get("wstatus"))

    is_closed   = wstatus.isin({"closed","complete","completed"})
    is_removal  = wotype.str.contains(r"\btree removal\b", na=False) | \
                  wotype.str.contains(r"\btree removal for tree planting\b", na=False) | \
                  wotype.str.contains(r"\btree down\b", na=False)
    is_planting = wotype.str.contains(r"\btree plant\b", na=False) | \
                  wotype.str.contains(r"\bplanting\b", na=False)

    w["is_tree_removal"]  = is_removal & is_closed
    w["is_tree_planting"] = is_planting & is_closed

    # Choose a best event date per row: prefer completion/closed → else earliest parsed date on row
    date_priority = [c for c in [
        "completeddate","closeddate","finishdate","completion_date","date_completed",
        "actual_end_date","actual_finish_date"
    ] if c in w.columns]
    dt_cols = [c for c in w.columns if pd.api.types.is_datetime64_any_dtype(w[c])]

    def pick_event_date(row):
        for c in date_priority:
            v = row.get(c)
            if pd.notna(v): return v
        vals = [row[c] for c in dt_cols if pd.notna(row[c])]
        return min(vals) if vals else pd.NaT

    w["event_date"] = w.apply(pick_event_date, axis=1)
    return w


####  Pulling in the Data from the NYC OPEN DATA API

In [ ]:
# --- Forestry Tree Points (all columns; keep only rows with geometry) ---
points_raw = socrata_fetch_tabular(
    DS_TREE_POINTS,
    where="location IS NOT NULL",
    select=None,
    limit=25000,
    max_rows=None
)
points = add_lat_lon_from_location(points_raw, "location").dropna(subset=["lat","lon"])
points = convert_datetime_columns(points, extra_force=("createddate", "updateddate"))
# -----------------------------------------------------------------------------------------------------
# Use 'createddate' as baseline start; we can refine with planting WOs later if earlier
if "createddate" not in points.columns:
    raise KeyError("Expected 'createddate' in Tree Points. Show points.columns if it's named differently.")

# Planting-space key in Tree Points
key_points = "plantingspaceglobalid" if "plantingspaceglobalid" in points.columns else \
             "planting_space_global_id" if "planting_space_global_id" in points.columns else None
if not key_points:
    raise KeyError("Expected a planting space id in Tree Points (e.g., 'plantingspaceglobalid').")

# --- Forestry Work Orders (all columns) ---
works_raw = socrata_fetch_tabular(
    DS_WORK_ORDERS,
    where=None,
    select=None,
    limit=25000,
    max_rows=None
)
works = convert_datetime_columns(works_raw)
works = classify_work_orders(works)

# Planting-space key in Work Orders
key_works = "plantingspaceglobalid" if "plantingspaceglobalid" in works.columns else \
            "planting_space_global_id" if "planting_space_global_id" in works.columns else None
if not key_works:
    print("⚠️ No planting-space key found in Work Orders; one-to-many join will be skipped.")

# --- 2015 Tree Census for enrichment (species, DBH cross-check, etc.) ---
# You can use this later for QA or to backfill attributes where Tree Points are missing.
census15 = socrata_fetch_tabular(
    DS_CENSUS_2015,
    where="latitude IS NOT NULL AND longitude IS NOT NULL",
    select=None,
    limit=25000,
    max_rows=200_000  # start smaller; set None to attempt full
)
census15 = convert_datetime_columns(census15)


#### Taking the Forestry Tree points data and limiting to the window of interest (2010 - 2017)

In [ ]:
cut_created_max = pd.Timestamp("2017-12-31")   # keep created <= this (exclude NaT and > 2017-12-31)
cut_planted_max = pd.Timestamp("2009-12-31")   # accept planted <= this OR NaT

### Assuptions made: Null values would imply older trees, while Newly planted trees are probably too. So essentially if they were planted in the window
# We ignore.

# --- Masks per your rules ---
# 1) CREATEDDATE: keep only rows with a non-null createddate on/before 2017-12-31
mask_created = points["createddate"].isna() | (points["createddate"] <= cut_created_max)

# 2) PLANTEDDATE: keep rows where planteddate is NaT OR planteddate <= 2009-12-31 We want trees planted beofre the window.
## Newly planted trees are probably too 
mask_planted = points["planteddate"].isna() | (points["planteddate"] <= cut_planted_max)

# --- Apply both filters ---
points_filtered = points.loc[mask_created & mask_planted].copy()

In [ ]:
#Only Alive Trees
points_filter = points_filtered[points_filtered['tpcondition']!='Dead']
points_filter = points_filter[~points_filter['tpstructure'].isin(['Stump - Uprooted','Stump'])]

In [ ]:
points_filter.head()

#### Starting with Work Order data for trees
- Limiting to those within the window, Not open, so closed (action taken), and using the data to find trees removed during the window.

In [ ]:
# 1) Pick the right status column and filter to CLOSED work orders
status_col = "wostatus" if "wostatus" in works_raw.columns else "wstatus"
closed_orders = works_raw.loc[
    works_raw[status_col].astype(str).str.lower().eq("closed")
].copy()

# 2) Parse dates
closed_orders["createddate"] = pd.to_datetime(closed_orders["createddate"], errors="coerce")
closed_orders["createddate"] = pd.to_datetime(closed_orders["createddate"], errors="coerce")
closed_orders["closeddate"] = pd.to_datetime(closed_orders["closeddate"], errors="coerce")


# 3) Keep rows with createddate < 2018-01-01 OR createddate is NaT
cut_off = pd.Timestamp("2018-01-01")
mask = ((closed_orders["createddate"].isna()) | (closed_orders["createddate"] < cut_off)
       &((closed_orders["createddate"].isna()) | (closed_orders["createddate"] < cut_off))
       &(closed_orders["closeddate"].isna() | (closed_orders["closeddate"] < cut_off)))
closed_orders_2017 = closed_orders.loc[mask].copy()
closed_orders_2017

print(len(closed_orders), "closed orders total")
print(len(closed_orders_2017), "closed orders with createddate < 2018-01-01 or NaT")

In [ ]:
### Looking at removals to flag via object ID in the forestry points data
removals = closed_orders_2017[((closed_orders_2017["wotype"].astype(str).str.lower().str.contains("removal"))|
    (closed_orders_2017["wocategory"].astype(str).str.lower().str.contains("removal")))]

In [ ]:
### saving files
# points_filter.to_csv("tree_points_filters.csv",index=False)
# removals.to_csv("tree_removals.csv",index=False)

In [ ]:
non_removed = points_filter[~points_filter['objectid'].isin(removals['objectid'].unique())]
print(non_removed.shape)
removed = points_filter[points_filter['objectid'].isin(removals['objectid'].unique())]
print(removed.shape)

#### Confirming that there are no duplicate tree listings in the work orders

In [ ]:
removed_lim = removed[["objectid","updateddate","createddate","riskratingdate"]].drop_duplicates()
#Checking object Id is 1:1 for removals
test = removed_lim.groupby(["objectid"]).agg({"updateddate":"count","createddate":"count"}).reset_index()
test["updateddate"][test["updateddate"]==0]=np.nan
test["entry_count"] = test["updateddate"].combine_first(test["createddate"])
test = test.drop(columns=["updateddate","createddate"])
test[test["entry_count"]>1]
## COnfirm no dupes

#### Using Contextual Ticket dates to identify Est. Removal Date. 
- Only interested in those here that were removed WITHIN the analysis window. 

In [ ]:
#Figuring out the Removed trees and when they were removed
# 1) Make sure the dates are real datetimes
for c in ["updateddate", "createddate"]:
    removed[c] = pd.to_datetime(removed[c], errors="coerce")

# 2) Removal date estimate: prefer updateddate, fall back to createddate
removed.loc[:, "removal_date_est"] = removed["updateddate"].fillna(removed["createddate"])
cutoff = pd.Timestamp("2018-01-01")  # everything strictly earlier than this
removed.loc[:, "removed_before_2018"] = removed["removal_date_est"] < cutoff

# Optional: keep only those rows
removed_2017_and_earlier = removed[removed["removed_before_2018"]]

## NO nulls, Checked
# removed[removed["Removal_Data_Est"].isnull()]


In [ ]:
print(removed.shape)
print(removed_2017_and_earlier.shape)

##### for those removals, adding flag to include or not incluide in analysis based on when during the eyar they were removed. 
- Those removed in June or before will not be included in year, but July and after included. Essentially if it was ardoun for majority of year - keep it. 

In [ ]:
removed_2017_and_earlier["removal_year"] =  removed_2017_and_earlier["removal_date_est"].dt.year
# 2) Include-in-year flag:
#    True  -> removed in July (7) or later
#    False -> removed in June (6) or earlier
#    NaT   -> treated as False (not included)
m = removed_2017_and_earlier["removal_date_est"].dt.month
removed_2017_and_earlier["include_in_year"] = (m >= 7) & m.notna()

#### Performing the join with the filtered Points data to get usable tree existance data

In [ ]:
removed_lim_join = removed_2017_and_earlier[["objectid","removal_date_est","removed_before_2018","removal_year","include_in_year"]]
trees_raw = points_filter.merge(removed_lim_join, how='left',on = ["objectid"] )

In [ ]:
##*** Analysis assumption is that the trees from this data set were around in 2010. ***
trees_raw["removed_before_2018"][trees_raw["removed_before_2018"].isnull()]=False
trees_raw

In [ ]:
trees_raw.shape

### Flattening the data long ways for each year in the analysis, removing the trees based on found removal work orders.

In [ ]:
## Processing the Tree Data to Have Yearly values for eahc tree:
trees_processing = []
trees_raw['manual_year'] = None
for year in range(2010,2018):
    print(year)
    trees_raw['manual_year'] = year
    trees_incl = trees_raw[((trees_raw["removal_year"].isnull())
                        |(trees_raw["removal_year"]>year)
                        |((trees_raw["removal_year"]==year) & (trees_raw["include_in_year"]==True)))]
    trees_processing.append(trees_incl)
    
    
    

In [ ]:
trees_processed = pd.concat(trees_processing)
# trees_processed.to_csv("trees_processed_for_part3.csv", index=False)